In [1]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.neural_network import MLPClassifier

In [2]:
import imagedataset as imgdat
import fishervector as fv

In [7]:
#Get Annotations
annotations = imgdat.GetAnnotations()

In [3]:
#Try to load all descriptors previously generated if not generate one
try:
    all_descriptors = imgdat.LoadAllDescriptors()
except:
    all_descriptors = imgdat.GenerateAllDescriptors(annotations)

Loading all sift descriptors in dataset from file...
All descriptors loaded ! 


In [4]:
#Use standardscaler for all SIFT descriptors before process them with any representation algorithm.
sc_dscs = StandardScaler()
all_descriptors = sc_dscs.fit_transform(all_descriptors)

In [5]:
#Clustering descriptors using KMeans in 120 classes 
try:
    gmm_fv = fv.LoadKmeans()
except:
    n_clusters = 20
    gmm_fv = fv.GenerateGmm(all_descriptors, n_clusters)

Training GMM of size 20
Initialization 0
  Iteration 0	 time lapse 44.91096s	 ll change inf
  Iteration 10	 time lapse 20.32084s	 ll change 1.45391
  Iteration 20	 time lapse 18.21832s	 ll change 0.35959
  Iteration 30	 time lapse 17.39839s	 ll change 0.13484
  Iteration 40	 time lapse 17.14599s	 ll change 0.11597
  Iteration 50	 time lapse 17.41200s	 ll change 0.05448
  Iteration 60	 time lapse 17.37600s	 ll change 0.01722
  Iteration 70	 time lapse 17.46901s	 ll change 0.01986
  Iteration 80	 time lapse 17.70833s	 ll change 0.05736
  Iteration 90	 time lapse 17.21199s	 ll change 0.00195
Initialization converged: True	 time lapse 210.50814s	 ll -85.63573
GaussianMixture(covariance_type='diag', init_params='kmeans', max_iter=100,
        means_init=None, n_components=20, n_init=1, precisions_init=None,
        random_state=None, reg_covar=1e-06, tol=0.001, verbose=2,
        verbose_interval=10, warm_start=False, weights_init=None)
Saving gmm at: means.gmm.npy, covs.gmm.npy, weights.gm

In [8]:
#Image representation
try:
    X_fv = np.load("fv_X.npy")
    y_fv = np.load("fv_y.npy")
except:
    X_fv, y_fv = fv.FisherFeatures(annotations, gmm_fv, sc_dscs)
    np.save("fv_X", X_fv)
    np.save("fv_y", y_fv)

No se pudo generar la region in : 009_image.png
No se pudo generar la region in : 018_image.png


In [9]:
#Classification 
#Dataset splitting into the training set and test set
X_train_fv, X_test_fv, y_train_fv, y_test_fv = train_test_split(X_fv, y_fv, test_size = 0.2, random_state = 0)

In [10]:
#Scaling representation features
sc_repr_fv = StandardScaler()
X_train_fv = sc_repr_fv.fit_transform(X_train_fv)
X_test_fv = sc_repr_fv.transform(X_test_fv)

In [11]:
#Using a MLP
n_layers = 2

for x in range (100,200):
    n_neurons = x
#n_neurons = 100
    clf_mlp = MLPClassifier(
        hidden_layer_sizes=np.ones(n_layers).astype(int)*n_neurons, 
        max_iter=1000,
        shuffle = True,
        alpha = 0.01)
    clf_mlp.fit(X_train_fv, y_train_fv)
    
    print('==========')
    print('score: '+str(clf_mlp.score(X_test_fv,y_test_fv)))

    y_pred = clf_mlp.predict(X_test_fv)
    y_pred = (y_pred > 0.5)
    cm = confusion_matrix(y_test_fv, y_pred)

    print('n_neurons: '+str(n_neurons))
    print(cm)

score: 0.949494949495
n_neurons: 100
[[34  4]
 [ 1 60]]
score: 0.929292929293
n_neurons: 101
[[33  5]
 [ 2 59]]
score: 0.919191919192
n_neurons: 102
[[34  4]
 [ 4 57]]
score: 0.949494949495
n_neurons: 103
[[34  4]
 [ 1 60]]
score: 0.919191919192
n_neurons: 104
[[33  5]
 [ 3 58]]
score: 0.929292929293
n_neurons: 105
[[33  5]
 [ 2 59]]
score: 0.919191919192
n_neurons: 106
[[32  6]
 [ 2 59]]
score: 0.969696969697
n_neurons: 107
[[37  1]
 [ 2 59]]
score: 0.939393939394
n_neurons: 108
[[35  3]
 [ 3 58]]
score: 0.909090909091
n_neurons: 109
[[30  8]
 [ 1 60]]
score: 0.949494949495
n_neurons: 110
[[33  5]
 [ 0 61]]
score: 0.929292929293
n_neurons: 111
[[33  5]
 [ 2 59]]
score: 0.919191919192
n_neurons: 112
[[33  5]
 [ 3 58]]
score: 0.929292929293
n_neurons: 113
[[33  5]
 [ 2 59]]
score: 0.919191919192
n_neurons: 114
[[33  5]
 [ 3 58]]
score: 0.929292929293
n_neurons: 115
[[32  6]
 [ 1 60]]
score: 0.949494949495
n_neurons: 116
[[34  4]
 [ 1 60]]
score: 0.949494949495
n_neurons: 117
[[35  3]
 [